In [1]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.identity import AzureCliCredential
from azure.ai.ml import automl, Input, MLClient
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.automl import (
    classification,
    ClassificationPrimaryMetrics,
    ClassificationModels,
)

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AzureML workspace
    subscription_id = ""
    resource_group = ""
    workspace = "workspacename"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

Found the config file in: /config.json


## Mostrar Espacio de trabajo

In [3]:
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)
subscription_id = ml_client.connections._subscription_id
resource_group = workspace.resource_group
workspace_name = ml_client.workspace_name
output = {}
output["Workspace"] = workspace_name
output["Subscription ID"] = subscription_id
output["Resource Group"] = resource_group
output["Location"] = workspace.location
output

{'Workspace': 'mltest',
 'Subscription ID': '8215b16a-6902-4007-b308-5d34c61892dc',
 'Resource Group': 'test_services',
 'Location': 'eastus'}

In [35]:
# Create MLTables for training dataset
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="./data/training-mltable-folder"
)

## Configuración del nombre del experimento

In [36]:
# General job parameters
compute_name = "mecastib1"
max_trials = 5
exp_name = "automlv2-Titanic-classifier-experiment"

In [37]:
# Create the AutoML classification job with the related factory-function.
classification_job = automl.classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name="Survived",
    primary_metric="accuracy",
    enable_model_explainability=True,
    tags={"my_custom_tag": "My Titanic Automl Exp"},
)
# Limits are all optional
classification_job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=max_trials,
    # max_concurrent_trials = 4,
    # max_cores_per_trial: -1,
    enable_early_termination=True,
)
# Training properties are optional
classification_job.set_training(
    blocked_training_algorithms=[ClassificationModels.LOGISTIC_REGRESSION],
    enable_onnx_compatible_models=True,
)

In [38]:
returned_job = ml_client.jobs.create_or_update(classification_job)
print(f"Created job: {returned_job}")


Uploading training-mltable-folder (0.06 MBs): 100%|██████████| 61623/61623 [00:00<00:00, 2720089.19it/s]




Created job: compute: azureml:mecastib1
creation_context:
  created_at: '2025-04-09T11:12:23.701471+00:00'
  created_by: Manuel Castiblanco
  created_by_type: User
display_name: ivory_spinach_c6zf8fd2b0
experiment_name: automlv2-Titanic-classifier-experiment
id: azureml:/subscriptions/8215b16a-6902-4007-b308-5d34c61892dc/resourceGroups/test_services/providers/Microsoft.MachineLearningServices/workspaces/mltest/jobs/ivory_spinach_c6zf8fd2b0
limits:
  enable_early_termination: true
  max_concurrent_trials: 1
  max_cores_per_trial: -1
  max_nodes: 1
  max_trials: 5
  timeout_minutes: 600
  trial_timeout_minutes: 20
log_verbosity: info
name: ivory_spinach_c6zf8fd2b0
outputs: {}
primary_metric: accuracy
properties: {}
queue_settings:
  job_tier: 'null'
resources:
  instance_count: 1
  shm_size: 2g
services:
  Studio:
    endpoint: https://ml.azure.com/runs/ivory_spinach_c6zf8fd2b0?wsid=/subscriptions/8215b16a-6902-4007-b308-5d34c61892dc/resourcegroups/test_services/workspaces/mltest&tid=07e

In [39]:
ml_client.jobs.stream(returned_job.name)

RunId: ivory_spinach_c6zf8fd2b0
Web View: https://ml.azure.com/runs/ivory_spinach_c6zf8fd2b0?wsid=/subscriptions/8215b16a-6902-4007-b308-5d34c61892dc/resourcegroups/test_services/workspaces/mltest

Execution Summary
RunId: ivory_spinach_c6zf8fd2b0
Web View: https://ml.azure.com/runs/ivory_spinach_c6zf8fd2b0?wsid=/subscriptions/8215b16a-6902-4007-b308-5d34c61892dc/resourcegroups/test_services/workspaces/mltest



In [40]:
# Get a URL for the status of the job
returned_job.services["Studio"].endpoint
print(returned_job.name)

ivory_spinach_c6zf8fd2b0


In [41]:
import mlflow
# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri
print(MLFLOW_TRACKING_URI)
# Set the MLFLOW TRACKING URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))

azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/8215b16a-6902-4007-b308-5d34c61892dc/resourceGroups/test_services/providers/Microsoft.MachineLearningServices/workspaces/mltest

Current tracking uri: azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/8215b16a-6902-4007-b308-5d34c61892dc/resourceGroups/test_services/providers/Microsoft.MachineLearningServices/workspaces/mltest


In [42]:
from mlflow.tracking.client import MlflowClient
# Initialize MLFlow client
mlflow_client = MlflowClient()
job_name = returned_job.name
# Example if providing an specific Job name/ID
# job_name = "b4e95546-0aa1-448e-9ad6-002e3207b4fc"
# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)
print("Parent Run: ")
print(mlflow_parent_run)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


Parent Run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.8645401412244574,
 'AUC_micro': 0.8760167487270125,
 'AUC_weighted': 0.8645401412244574,
 'accuracy': 0.8193008739076154,
 'average_precision_score_macro': 0.8557024916458076,
 'average_precision_score_micro': 0.8627919256593868,
 'average_precision_score_weighted': 0.8669185465969991,
 'balanced_accuracy': 0.7994686963353901,
 'f1_score_macro': 0.8021751643949143,
 'f1_score_micro': 0.8193008739076154,
 'f1_score_weighted': 0.8169519134697142,
 'log_loss': 0.4368755522620926,
 'matthews_correlation': 0.6140198582743006,
 'norm_macro_recall': 0.59893739267078,
 'precision_score_macro': 0.8153219506029398,
 'precision_score_micro': 0.8193008739076154,
 'precision_score_weighted': 0.8240178808074334,
 'recall_score_macro': 0.7994686963353901,
 'recall_score_micro': 0.8193008739076154,
 'recall_score_weighted': 0.8193008739076154,
 'weighted_accuracy': 0.8361920807411917}, params={}, tags={'automl_best_child_run_id': 'ivory_spinach

In [43]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

{'my_custom_tag': 'My Titanic Automl Exp', 'model_explain_run': 'best_run', 'pipeline_id_000': '5dfac790c5c209f98a1da2dc1c7fb76f0397324f;c7af0367625be6ac5c2fecbfc72ed444cb7a2111;799d2168db11fc19b9e1c6c1df62f8981ad39fe9;__AutoML_Ensemble__;__AutoML_Stack_Ensemble__', 'score_000': '0.8114731585518102;0.7755430711610487;0.7867665418227217;0.8170661672908865;0.8193008739076154', 'predicted_cost_000': '0;0;0.5;0;0', 'fit_time_000': '0.0799936;0.47903450000000003;0.2670052;4;6', 'training_percent_000': '100;100;100;100;100', 'iteration_000': '0;1;2;3;4', 'run_preprocessor_000': 'MaxAbsScaler;MaxAbsScaler;MaxAbsScaler;;', 'run_algorithm_000': 'LightGBM;XGBoostClassifier;ExtremeRandomTrees;VotingEnsemble;StackEnsemble', 'automl_best_child_run_id': 'ivory_spinach_c6zf8fd2b0_4', 'mlflow.rootRunId': 'ivory_spinach_c6zf8fd2b0', 'mlflow.runName': 'ivory_spinach_c6zf8fd2b0', 'mlflow.user': 'Manuel Castiblanco'}


In [44]:
# Get the best model's child run
best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)
best_run = mlflow_client.get_run(best_child_run_id)
print("Best child run: ")
print(best_run)

In [45]:
best_run.data.metrics


{'norm_macro_recall': 0.59893739267078,
 'f1_score_weighted': 0.8169519134697142,
 'weighted_accuracy': 0.8361920807411917,
 'AUC_macro': 0.8645401412244574,
 'matthews_correlation': 0.6140198582743006,
 'recall_score_macro': 0.7994686963353901,
 'AUC_micro': 0.8760167487270125,
 'recall_score_weighted': 0.8193008739076154,
 'log_loss': 0.4368755522620926,
 'f1_score_micro': 0.8193008739076154,
 'average_precision_score_micro': 0.8627919256593868,
 'recall_score_micro': 0.8193008739076154,
 'average_precision_score_macro': 0.8557024916458076,
 'AUC_weighted': 0.8645401412244574,
 'f1_score_macro': 0.8021751643949143,
 'balanced_accuracy': 0.7994686963353901,
 'average_precision_score_weighted': 0.8669185465969991,
 'precision_score_weighted': 0.8240178808074334,
 'accuracy': 0.8193008739076154,
 'precision_score_macro': 0.8153219506029398,
 'precision_score_micro': 0.8193008739076154}

In [46]:
import os
# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)
# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(
    best_run.info.run_id, "outputs", local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))
# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

Artifacts downloaded in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/mecastib1/code/Users/mecastib/artifact_downloads/outputs
Artifacts: ['conda_env_v_1_0_0.yml', 'engineered_feature_names.json', 'env_dependencies.json', 'featurization_summary.json', 'generated_code', 'internal_cross_validated_models.pkl', 'mlflow-model', 'model.pkl', 'pipeline_graph.json', 'run_id.txt', 'scoring_file_pbi_v_1_0_0.py', 'scoring_file_v_1_0_0.py', 'scoring_file_v_2_0_0.py']


['conda.yaml', 'MLmodel', 'model.pkl', 'python_env.yaml', 'requirements.txt']

In [68]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)
from azure.ai.ml.constants import ModelType
model_name = "titanic-model-v2"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/model.pkl",
    name=model_name,
    description="my sample titanic mlflow model",
)
# for downloaded file
# model = Model(path="artifact_downloads/outputs/model.pkl", name=model_name)
registered_model = ml_client.models.create_or_update(model)
registered_model.id


'/subscriptions/8215b16a-6902-4007-b308-5d34c61892dc/resourceGroups/test_services/providers/Microsoft.MachineLearningServices/workspaces/mltest/models/titanic-model-v2/versions/4'

In [79]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime
online_endpoint_name = "titanic-dev-" + datetime.datetime.now().strftime("%m%d%H%M%f")
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for titanic mlflow model",
    auth_mode="key",
    tags={"foo": "bar"},
)
ml_client.begin_create_or_update(endpoint)

In [81]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import uuid

endpoint_name = "titanic-endpoint-" + uuid.uuid4().hex[:6]

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Clean endpoint for Titanic AutoML model",
    auth_mode="key"
)

ml_client.begin_create_or_update(endpoint).result()
print(f"✅ Endpoint creado: {endpoint_name}")


HttpResponseError: (SubscriptionNotRegistered) Resource provider [N/A] isn't registered with Subscription [N/A]. Please see troubleshooting guide, available here: https://aka.ms/register-resource-provider
Code: SubscriptionNotRegistered
Message: Resource provider [N/A] isn't registered with Subscription [N/A]. Please see troubleshooting guide, available here: https://aka.ms/register-resource-provider

In [75]:
env = Environment(
    name="automl-titanic-tabular-env",
    description="environment for automl inference",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1",
    conda_file="artifact_downloads/outputs/conda_env_v_1_0_0.yml",
)

In [76]:
code_configuration = CodeConfiguration(
    code="artifact_downloads/outputs/", scoring_script="scoring_file_v_2_0_0.py"
)

In [77]:
deployment = ManagedOnlineDeployment(
    name="titanic-deploy",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    environment=env,
    code_configuration=code_configuration,
    instance_type="Standard_DS2_V2",
    instance_count=1,
)

In [78]:
ml_client.online_deployments.begin_create_or_update(deployment)


Instance type Standard_DS2_V2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint titanic-04091153557528 exists


HttpResponseError: (BadRequest) The request is invalid.
Code: BadRequest
Message: The request is invalid.
Exception Details:	(InferencingClientCallFailed) {"error":{"code":"Validation","message":"{\"errors\":{\"\":[\"This endpoint has not been created successfully or is in deleting provisioning state. Please recreate endpoint and then try again to create deployment.\"]},\"type\":\"https://tools.ietf.org/html/rfc9110#section-15.5.1\",\"title\":\"One or more validation errors occurred.\",\"status\":400,\"traceId\":\"00-ebc3dc50d9320cffbbc992aa2d24f8d0-28afaf69c437074f-01\"}"}}
	Code: InferencingClientCallFailed
	Message: {"error":{"code":"Validation","message":"{\"errors\":{\"\":[\"This endpoint has not been created successfully or is in deleting provisioning state. Please recreate endpoint and then try again to create deployment.\"]},\"type\":\"https://tools.ietf.org/html/rfc9110#section-15.5.1\",\"title\":\"One or more validation errors occurred.\",\"status\":400,\"traceId\":\"00-ebc3d